In [11]:
#importing packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from datetime import datetime
import csv
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [12]:
#extracting urls from web page
req = Request('https://www.deutsche-startups.de/verzeichnisse/startups-a-z/')
html_page = urlopen(req)
#Generating soup
soup = BeautifulSoup(html_page, "lxml")
#extracting links from page
links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [13]:
#filtering the urls that correspond to startups
#getting rid of werbung and other links
clean_links = links[130:1212]
len(clean_links)

1082

In [14]:
#Extracting main info from a startup webpage
url1 = clean_links[0]
res1 = requests.get(url1)
soup = BeautifulSoup(res1.text, 'html.parser')
#getting rid of the <span>
for match in soup.findAll('span'):
    match.unwrap()
cards =soup.find_all('div', {'id': 'contentMain'})


In [ ]:
# generating card from a URL
def get_info(card):
    card = cards[0]
    ultag = card.div.ul
    atag = card.div.a
    company_name = ultag.find('li', 'company').getText().split(':')[1]
    company_url = atag.get('href')
    company_head = ultag.find('li', 'head').getText().split(':')[1]
    founded_year = ultag.find('li', 'founded').getText().split(':')[1]
    location = ultag.find('li', 'location').getText().split(':')[1]
    model = ultag.find('li', 'businessmodel').getText().split(':')[1]
    contact = ultag.find('li', 'contact').getText().split(':')[1].replace(' (at) ', '@').replace(' (punkt) ', '.')
    info = (company_name, company_url, company_head, founded_year, location, model, contact)
    return info
    




In [20]:
#generating cards from all urls
def get_info_new(url):
    res1 = requests.get(url)
    soup = BeautifulSoup(res1.text, 'html.parser')
    for match in soup.findAll('span'):
        match.unwrap()
    cards =soup.find_all('div', {'id': 'contentMain'})
    card = cards[0]
    ultag = card.div.ul
    atag = card.div.a
    company_name = ultag.find('li', 'company').getText().split(':')[1]
    try:
        company_url = atag.get('href')
    except AttributeError:
        company_url = ''
    try:
        company_head = ultag.find('li', 'head').getText().split(':')[1]
    except AttributeError:
            company_head = ''
    try:
        founded_year = ultag.find('li', 'founded').getText().split(':')[1]
    except AttributeError:
        founded_year =''
    try:
        location = ultag.find('li', 'location').getText().split(':')[1]
    except AttributeError:
        location = ''
    try:
        model = ultag.find('li', 'businessmodel').getText().split(':')[1]
    except AttributeError:
        model = ''
    try:
        contact = ultag.find('li', 'contact').getText().split(':')[1].replace(' (at) ', '@').replace(' (punkt) ', '.')
    except AttributeError:
        contact = ''
    
    info = (company_name, company_url, company_head, founded_year, location, model, contact)
    return info

In [21]:
# appending into new list
infos = []
for link in clean_links:
    infos.append(get_info_new(link))

In [25]:
#Generating CSV file from data
with open('deutsche_startups.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Company_name', 'URL', 'company_head', 'founded_year', 'location', 'model', 'contact'])
        writer.writerows(infos)

In [26]:
startups = pd.read_csv('deutsche_startups.csv')
startups.head()

,Company_name,URL,company_head,founded_year,location,model,contact
0,1000Mikes AG,http://www.1000mikes.com/,"Frank Felix Debatin, Peter Schweyer",2007,Hamburg,User generated Webradio,info@1000mikes.com
1,12designer,http://www.12designer.com,Lars Bamberg,2009,Berlin,Marktplatz für Kreatives,NaN
2,12like GmbH,http://www.12like.com,"Kathrin Anselm, Dirk Wischnewski",2010,Berlin,Partnervermittlung,NaN
3,15Talents.com GmbH & Co. KG,http://www.15talents.com/,"Marc Dr?ner, Henrike Fr?chling",2008,Berlin,Projektmartktplatz für Unternehmen und Studenten,team@15talents.com
4,21Publish Web Services GmbH,http://www.yourteam.de/,Stefan Wiskemann,2007,Hamburg,Mannschafts-Community,NaN
